In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import platform
import random
import uuid
import os
import os.path
import skimage
import sak
import sak.wavelet
import sak.data
import sak.data.augmentation
import numpy as np
import scipy as sp
import scipy.signal
import pandas as pd
import networkx
import networkx.algorithms.approximation
import wfdb
import json
import tqdm
import matplotlib.pyplot as plt
from scipy.stats import norm
from sak.signal import StandardHeader

# Data loader to un-clutter code    
def load_data(filepath):
    dic = dict()
    with open(filepath) as f:
        text = list(f)
    for line in text:
        line = line.replace(' ','').replace('\n','').replace(',,','')
        if line[-1] == ',': line = line[:-1]
        head = line.split(',')[0]
        tail = line.split(',')[1:]
        if tail == ['']:
            tail = np.asarray([])
        else:
            tail = np.asarray(tail).astype(int)

        dic[head] = tail
    return dic


def trailonset(sig,on):
    on = on-sig[0]
    off = on-sig[0]+sig[-1]
    sig = sig+np.linspace(on,off,sig.size)
    
    return sig

def getcorr(segments):
    if len(segments) > 0:
        length = 2*max([segments[i][2].size for i in range(len(segments))])
    else:
        return np.zeros((0,0))

    corr = np.zeros((len(segments),len(segments)))

    for i in range(len(segments)):
        for j in range(len(segments)):
            if i != j:
                if segments[i][2].size != segments[j][2].size:
                    if segments[i][2].size != 1:
                        x1 = sp.interpolate.interp1d(np.linspace(0,1,len(segments[i][2])),segments[i][2])(np.linspace(0,1,length))
                    else:
                        x1 = np.full((length,),segments[i][2][0])
                    if segments[j][2].size != 1:
                        x2 = sp.interpolate.interp1d(np.linspace(0,1,len(segments[j][2])),segments[j][2])(np.linspace(0,1,length))
                    else:
                        x2 = np.full((length,),segments[j][2][0])
                else:
                    x1 = segments[i][2]
                    x2 = segments[j][2]
                if (x1.size == 1) and (x2.size == 1):
                    corr[i,j] = 1
                else:
                    c,_ = sak.signal.xcorr(x1,x2)
                    corr[i,j] = np.max(np.abs(c))
            else:
                corr[i,j] = 1
                
    return corr

def getdelete(segments, threshold):
    corr = getcorr(segments)
    
    index_delete = []
    
    for i in range(corr.shape[0]):
        if i in index_delete:
            continue
        for j in range(corr.shape[1]):
            if j == i:
                continue
            if corr[i,j] > threshold:
                if j not in index_delete:
                    index_delete.append(j)
                
    return index_delete
    

In [3]:
if platform.system() in ['Linux', 'Linux2']:
    basedir = '/media/guille/DADES/DADES/Delineator'
else:
    basedir = r'C:\Users\Emilio\Documents\DADES\DADES\Delineator'

In [4]:
Pamplitudes = {}
QRSamplitudes = {}
Tamplitudes = {}

for i in tqdm.tqdm(range(200)):
    (signal, header) = wfdb.rdsamp(os.path.join(basedir,'ludb','{}'.format(i+1)))
    sortOrder = np.where(np.array([x.upper() for x in header['sig_name']])[:,None] == StandardHeader)[1]
    signal = signal[:,sortOrder]
    if header['fs'] != 500:
        print(header['fs'])
    signal = sp.signal.decimate(signal,2,axis=0)
    
    # 1st step: reduce noise
    signal = sp.signal.filtfilt(*sp.signal.butter(4,   0.5/250., 'high'),signal.T).T
    signal = sp.signal.filtfilt(*sp.signal.butter(4, 125.0/250.,  'low'),signal.T).T
    
    Psegments   = [[] for _ in StandardHeader]
    QRSsegments = [[] for _ in StandardHeader]
    Tsegments   = [[] for _ in StandardHeader]

    # 2nd step: retrieve onsets and offsets
    for j in range(len(StandardHeader)):
        lead = StandardHeader[j]
        name = str(i+1)+"_"+lead
        ann = wfdb.rdann(os.path.join(basedir,'ludb','{}'.format(i+1)),'atr_{}'.format(lead.lower()))
        
        locP = np.where(np.array(ann.symbol) == 'p')[0]
        if len(locP) != 0:
            if locP[0]-1 < 0:
                locP = locP[1:]
            if locP[-1]+1 == len(ann.sample):
                locP = locP[:-1]
        Pon[name] = ann.sample[locP-1]//2
        Ppeak[name] = ann.sample[locP]//2
        Poff[name] = ann.sample[locP+1]//2

        locQRS = np.where(np.array(ann.symbol) == 'N')[0]
        if len(locQRS) != 0:
            if locQRS[0]-1 < 0:
                locQRS = locQRS[1:]
            if locQRS[-1]+1 == len(ann.sample):
                locQRS = locQRS[:-1]
        QRSon[name] = ann.sample[locQRS-1]//2
        QRSpeak[name] = ann.sample[locQRS]//2
        QRSoff[name] = ann.sample[locQRS+1]//2
        
        for k in range(len(locQRS)):
            QRSsegments[j].append

        locT = np.where(np.array(ann.symbol) == 't')[0]
        if len(locT) != 0:
            if locT[0]-1 < 0:
                locT = locT[1:]
            if locT[-1]+1 == len(ann.sample):
                locT = locT[:-1]
        Ton[name] = ann.sample[locT-1]//2
        Tpeak[name] = ann.sample[locT]//2
        Toff[name] = ann.sample[locT+1]//2
        
        # Store group
        group[name] = str(i+1)
        agakjdsgklj

dataset = pd.DataFrame(dataset)

  0%|          | 0/200 [00:00<?, ?it/s]


NameError: name 'agakjdsgklj' is not defined

In [10]:
signal.shape

(2500, 12)

In [7]:
ann.sample[locQRS-1]//2

array([ 320,  662,  990, 1312, 1648, 1976])

In [8]:
ann.sample[locP-1]//2

array([ 626,  955, 1269, 1612, 1941])

In [9]:
ann.sample[locT-1]//2

array([ 386,  728, 1059, 1379, 1717])

{'1_I': array([ 320,  662,  990, 1312, 1648, 1976])}